In [1]:
using Graphs

g = simple_graph(5)

inputs = [       # each element is (u, v, dist)
    (1, 2, 10.),
    (1, 3, 5.),
    (2, 3, 2.),
    (3, 2, 3.),
    (2, 4, 1.),
    (3, 5, 2.),
    (4, 5, 4.),
    (5, 4, 6.),
    (5, 1, 7.),
    (3, 4, 9.) ]

ne = length(inputs)
dists = zeros(ne)

for i = 1 : ne
    a = inputs[i]
    add_edge!(g, a[1], a[2])   # add edge
    dists[i] = a[3]             # set distance
end

r = dijkstra_shortest_paths(g, dists, 1)

d = 4
println(r.dists[d])
println(enumerate_paths(vertices(g), r.parent_indices, d))

;

LoadError: LoadError: ArgumentError: module Graphs not found in current path.
Run `Pkg.add("Graphs")` to install the Graphs package.
while loading In[1], in expression starting on line 1

In [6]:
using JLD

push!(LOAD_PATH, ".")
using Util


function policy2name(tree_policy)
    
    scale_str = ""
    if haskey(tree_policy, "bScale") && tree_policy["bScale"] == true
        scale_str = "s"
    end
    
    if tree_policy["type"] == :UCB1
        if haskey(tree_policy, "c")
            return string(tree_policy["type"]) * scale_str * "(" * string(tree_policy["c"]) * ")"
        else
            return string(tree_policy["type"]) * scale_str
        end
    elseif tree_policy["type"] == :TS || tree_policy["type"] == :TSM
        return string(tree_policy["type"])
    elseif tree_policy["type"] == :AUCB
        sp = ""
        for i = 1:length(tree_policy["SP"])
            sp *= policy2name(tree_policy["SP"][i])
            if i != length(tree_policy["SP"])
                sp *= ","
            end
        end
        if haskey(tree_policy, "CP")
            cp = policy2name(tree_policy["CP"])
        else
            cp = "TSN"
        end
        return "AUCB" * scale_str * "(" * cp * ")(" * sp * ")"
    elseif tree_policy["type"] == :UCB1withCE
        return "UCB1(CE)"
    end
end


function processData(datafile::ASCIIString; datadir::ASCIIString = ".")
    
    D = load(joinpath(datadir, datafile));
    
    Scenarios = D["Scenarios"]
    TreePolicies = D["TreePolicies"]
    Results = D["Results"]
    
    R = Dict()
    
    for (key, results) in Results
        scenario, tree_policy = key
        expected_returns = results["expected_returns"]
        if !haskey(R, scenario)
            R[scenario] = Dict()
        end
        R[scenario]["optimum"] = results["opt_dist"]
        tpn = policy2name(tree_policy)
        R[scenario][tpn] = Dict()
        R[scenario][tpn]["mean"] = mean(expected_returns)
        R[scenario][tpn]["std"] = std(expected_returns)
    end

    for scenario in Scenarios
        R_ = R[scenario]
        println("Scenario: ", scenario, ", Optimum: ", neat(R_["optimum"]))
        for tree_policy in TreePolicies[scenario]
            tpn = policy2name(tree_policy)
            println("  policy: ", @sprintf("%-34s", tpn), ", mean: ", @sprintf("%7s", neat(R_[tpn]["mean"])), ", std: ", @sprintf("%6.2f", neat(R_[tpn]["std"])))
        end
        println()
    end
end

;

In [146]:
processData("exp_conv.jld", datadir = "data01")

Scenario: 26229, Optimum: -754.6
  policy: UCB1(1.0)                       , mean:  -959.8, std: 164.30
  policy: UCB1(300.0)                     , mean:  -821.3, std:  70.45
  policy: TS                              , mean:  -764.6, std:  18.34
  policy: TSM                             , mean:  -763.8, std:  15.86
  policy: AUCB(TSN)(UCB1(1.0),UCB1(300.0)), mean:  -787.4, std:  46.15
  policy: AUCB(TSN)(UCB1(1.0),TSM)        , mean:  -766.6, std:  20.26

Scenario: 13157, Optimum: -841.4
  policy: UCB1(1.0)                       , mean: -1005.0, std: 140.60
  policy: UCB1(300.0)                     , mean:  -889.1, std:  71.15
  policy: TS                              , mean:  -849.3, std:  21.86
  policy: TSM                             , mean:  -848.8, std:  21.66
  policy: AUCB(TSN)(UCB1(1.0),UCB1(300.0)), mean:  -870.0, std:  50.60
  policy: AUCB(TSN)(UCB1(1.0),TSM)        , mean:  -857.1, std:  34.69

Scenario: 10978, Optimum: -858.9
  policy: UCB1(1.0)                       , mea

In [147]:
processData("exp_1000.jld", datadir = "data01")

Scenario: 26229, Optimum: -754.6
  policy: UCB1(1.0)                       , mean:  -959.2, std: 164.40
  policy: UCB1(300.0)                     , mean:  -833.3, std:  74.34
  policy: TS                              , mean:  -771.4, std:  24.38
  policy: TSM                             , mean:  -774.0, std:  24.11
  policy: AUCB(TSN)(UCB1(1.0),UCB1(300.0)), mean:  -796.3, std:  46.60
  policy: AUCB(TSN)(UCB1(1.0),TSM)        , mean:  -779.4, std:  28.49

Scenario: 13157, Optimum: -841.4
  policy: UCB1(1.0)                       , mean: -1005.0, std: 140.60
  policy: UCB1(300.0)                     , mean:  -897.9, std:  81.57
  policy: TS                              , mean:  -852.2, std:  27.23
  policy: TSM                             , mean:  -850.1, std:  25.34
  policy: AUCB(TSN)(UCB1(1.0),UCB1(300.0)), mean:  -869.7, std:  51.24
  policy: AUCB(TSN)(UCB1(1.0),TSM)        , mean:  -856.6, std:  34.13

Scenario: 10978, Optimum: -858.9
  policy: UCB1(1.0)                       , mea

In [149]:
processData("exp_100.jld", datadir = "data01")

Scenario: 26229, Optimum: -754.6
  policy: UCB1(1.0)                       , mean:  -971.0, std: 176.80
  policy: UCB1(300.0)                     , mean:  -869.6, std: 117.70
  policy: TS                              , mean:  -801.9, std:  51.07
  policy: TSM                             , mean:  -805.8, std:  43.91
  policy: AUCB(TSN)(UCB1(1.0),UCB1(300.0)), mean:  -815.6, std:  56.37
  policy: AUCB(TSN)(UCB1(1.0),TSM)        , mean:  -817.6, std:  64.56

Scenario: 13157, Optimum: -841.4
  policy: UCB1(1.0)                       , mean: -1007.0, std: 141.90
  policy: UCB1(300.0)                     , mean:  -927.4, std: 114.90
  policy: TS                              , mean:  -867.7, std:  42.30
  policy: TSM                             , mean:  -870.2, std:  49.43
  policy: AUCB(TSN)(UCB1(1.0),UCB1(300.0)), mean:  -890.8, std:  69.32
  policy: AUCB(TSN)(UCB1(1.0),TSM)        , mean:  -878.5, std:  67.95

Scenario: 10978, Optimum: -858.9
  policy: UCB1(1.0)                       , mea

In [162]:
processData("exp_param_0.jld", datadir = "data01")

Scenario: 26229, Optimum: -209.4
  policy: UCB1(1.0)                       , mean:  -213.4, std:   8.42
  policy: UCB1(10.0)                      , mean:  -211.8, std:   6.91
  policy: UCB1(20.0)                      , mean:  -210.5, std:   3.69
  policy: UCB1(30.0)                      , mean:  -209.9, std:   2.46
  policy: UCB1(40.0)                      , mean:  -209.6, std:   1.44
  policy: UCB1(50.0)                      , mean:  -209.6, std:   1.44
  policy: UCB1(60.0)                      , mean:  -209.4, std:   0.00
  policy: UCB1(70.0)                      , mean:  -209.4, std:   0.00
  policy: UCB1(80.0)                      , mean:  -209.4, std:   0.00
  policy: UCB1(90.0)                      , mean:  -209.4, std:   0.00
  policy: UCB1(100.0)                     , mean:  -209.4, std:   0.00

Scenario: 13157, Optimum: -216.0
  policy: UCB1(1.0)                       , mean:  -233.7, std:  12.47
  policy: UCB1(10.0)                      , mean:  -225.5, std:   8.11
  policy: 

In [166]:
processData("exp_param_0__0_2.jld", datadir = "data01")

Scenario: 26229, Optimum: -754.6
  policy: UCB1(1.0)                       , mean:  -959.8, std: 164.30
  policy: UCB1(100.0)                     , mean:  -914.4, std: 162.60
  policy: UCB1(200.0)                     , mean:  -874.1, std: 143.50
  policy: UCB1(300.0)                     , mean:  -821.3, std:  70.45
  policy: UCB1(400.0)                     , mean:  -793.6, std:  61.29
  policy: UCB1(500.0)                     , mean:  -777.6, std:  30.07
  policy: UCB1(600.0)                     , mean:  -771.0, std:  24.48
  policy: UCB1(700.0)                     , mean:  -762.0, std:  15.14
  policy: UCB1(800.0)                     , mean:  -764.7, std:  16.26
  policy: UCB1(900.0)                     , mean:  -761.7, std:  14.41
  policy: UCB1(1000.0)                    , mean:  -764.2, std:  16.00
  policy: UCB1(1100.0)                    , mean:  -760.7, std:  13.68
  policy: UCB1(1200.0)                    , mean:  -762.8, std:  15.16
  policy: UCB1(1300.0)                    , 

In [173]:
processData("exp_param_0_2.jld", datadir = "data01")

Scenario: 26229, Optimum: -1968.0
  policy: UCB1(1.0)                       , mean: -2009.0, std:  22.91
  policy: UCB1(100.0)                     , mean: -2010.0, std:  23.88
  policy: UCB1(200.0)                     , mean: -2003.0, std:  21.55
  policy: UCB1(300.0)                     , mean: -1996.0, std:  19.44
  policy: UCB1(400.0)                     , mean: -1994.0, std:  20.59
  policy: UCB1(500.0)                     , mean: -1990.0, std:  16.50
  policy: UCB1(600.0)                     , mean: -1984.0, std:  14.17
  policy: UCB1(700.0)                     , mean: -1983.0, std:  15.06
  policy: UCB1(800.0)                     , mean: -1983.0, std:  15.85
  policy: UCB1(900.0)                     , mean: -1984.0, std:  15.69
  policy: UCB1(1000.0)                    , mean: -1982.0, std:  12.25
  policy: UCB1(1100.0)                    , mean: -1984.0, std:  14.83
  policy: UCB1(1200.0)                    , mean: -1982.0, std:  13.84
  policy: UCB1(1300.0)                    ,

In [263]:
processData("exp_100.jld", datadir = "data02")

Scenario: 26229, Optimum: -570.4
  policy: UCB1(50.0)                        , mean: -1087.0, std: 407.20
  policy: UCB1(10000.0)                     , mean:  -843.8, std: 233.90
  policy: TS                                , mean:  -937.4, std: 364.60
  policy: TSM                               , mean:  -774.9, std: 207.20
  policy: AUCB(TSN)(UCB1(50.0),UCB1(10000.0)), mean:  -785.8, std: 201.00
  policy: AUCB(TSN)(UCB1(50.0),TSM)         , mean:  -777.7, std: 205.30

Scenario: 13157, Optimum: -389.1
  policy: UCB1(50.0)                        , mean:  -422.2, std: 100.10
  policy: UCB1(10000.0)                     , mean:  -419.0, std:  68.92
  policy: TS                                , mean:  -464.5, std: 146.20
  policy: TSM                               , mean:  -407.8, std:  66.13
  policy: AUCB(TSN)(UCB1(50.0),UCB1(10000.0)), mean:  -396.5, std:  45.64
  policy: AUCB(TSN)(UCB1(50.0),TSM)         , mean:  -393.9, std:  39.65

Scenario: 10978, Optimum: -387.3
  policy: UCB1(50.0) 

In [270]:
processData("exp_1000.jld", datadir = "data02")

Scenario: 26229, Optimum: -570.4
  policy: UCB1(100.0)                       , mean: -1027.0, std: 416.40
  policy: UCB1(10000.0)                     , mean:  -718.2, std:  85.91
  policy: TS                                , mean:  -709.8, std: 219.10
  policy: TSM                               , mean:  -616.9, std:  64.74
  policy: AUCB(TSN)(UCB1(100.0),UCB1(10000.0)), mean:  -655.7, std:  97.38
  policy: AUCB(TSN)(UCB1(100.0),TSM)        , mean:  -649.0, std: 106.20

Scenario: 13157, Optimum: -389.1
  policy: UCB1(100.0)                       , mean:  -440.8, std: 115.40
  policy: UCB1(10000.0)                     , mean:  -390.5, std:   4.92
  policy: TS                                , mean:  -419.6, std:  87.64
  policy: TSM                               , mean:  -389.4, std:   0.61
  policy: AUCB(TSN)(UCB1(100.0),UCB1(10000.0)), mean:  -389.9, std:   0.76
  policy: AUCB(TSN)(UCB1(100.0),TSM)        , mean:  -389.6, std:   0.71

Scenario: 10978, Optimum: -387.3
  policy: UCB1(100.